#### 1. Подсчитать GC-content для гена с идентификатором X53950.1 (последовательность взять в GeneBank).

In [1]:
seq = ''.join(open('1task.txt').read().split('\n')[1:])
res = (seq.count('G') + seq.count('C')) / len(seq)
print(res)

0.35732133933033483


#### 2. Получить последовательность fasta из pdb файла (файл – frompdb.pdb), каждая цепь (последовательность) записывается отдельно со своим заголовком.

In [10]:
# http://cupnet.net/pdb2fasta/
TRANSITION_MAP = {'ALA':'A','ARG':'R','ASN':'N','ASP':'D','CYS':'C','GLU':'E','GLN':'Q','GLY':'G','HIS':'H', 'ILE':'I','LEU':'L','LYS':'K','MET':'M','PHE':'F','PRO':'P','SER':'S','THR':'T','TRP':'W', 'TYR':'Y','VAL':'V'}
seqs = [[]]
for line in open('frompdb.pdb'):
    toks = line.split()
    if toks[0] == 'ATOM' and toks[2] == 'CA':
        seqs[-1].append(toks[3])
    if toks[0] == 'TER':
        seqs.append([])
seqs.pop(-1)
for i, seq in enumerate(seqs):
    seqs[i] = ''.join(map(lambda s: TRANSITION_MAP[s], seq))
    print('>\n', seqs[i], sep='')

>
ADVQQKLAELERQSGGRLGVALINTADNSQILYRADERFPMCSTSKVMAAAAVLKQSETQKQLLNQPVEIKPADLVNYNPIAEKHVNGTMTLAELSAAALQYSDNTAMNKLIAQLGGPGGVTAFARAIGDETFRLDRTEPTLNTAIPGDPRDTTTPRAMAQTLRQLTLGHALGETQRAQLVTWLKGNTTGAASIRAGLPASWVVGDKTGSGGYGTTNDIAVIWPKDRAPLILVTYFTQPQPKAESRRDVLASAAKIVTDG
>
ADVQQKLAELERQSGGRLGVALINTADNSQILYRADERFPMCSTSKVMAAAAVLKQSETQKQLLNQPVEIKPADLVNYNPIAEKHVNGTMTLAELSAAALQYSDNTAMNKLIAQLGGPGGVTAFARAIGDETFRLDRTEPTLNTAIPGDPRDTTTPRAMAQTLRQLTLGHALGETQRAQLVTWLKGNTTGAASIRAGLPASWVVGDKTGSGGYGTTNDIAVIWPKDRAPLILVTYFTQPQPKAESRRDVLASAAKIVTDG


#### 3. Перевести ДНК последовательность в РНК, а потом в белок (файл dna.txt)

In [3]:
GENCODE = {'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R', 'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 'TAC':'Y', 'TAT':'Y', 'TAA':'*', 'TAG':'*', 'TGC':'C', 'TGT':'C', 'TGA':'*', 'TGG':'W'}
seq = open('dna.txt').read().strip()
pep_seq = ''.join([GENCODE[seq[i:i+3]] for i in range(0, len(seq) - 2, 3)])
print(pep_seq)

GMAITPRFYF*RRKVLT*ALPAQGPKKSPISYFRMTCVSLRVNHRPQFIEAWGNR*V*LA*ESKSWDHSVVTINLRWGFFGGFLQLPTRRFEVNQLRI*PRYPVISKLKHTVP*RLELLTGLFHACAIPPHSPAYPSERRQCDPPLRYSYV*RSYVFCRAGERVEHFTDGRDSGKGRIIGD*HRRRLRWRQLNRSSSALNNVLISTHSRQSTERLDYQRLSSSSNLLPAS**PPRD***LSAQN*DGADG*RTFGQPQFPTGQVLRCASL*MYKQPKWAEPGLSWFLRELETRDDSSLNIERGRRTVEKVIVPRVPTYSGYCLKLYYFRGGALKVSSSHD*TREGREVGSFNG*KTKAYCAQRNAHLAAGVLNARSPLSFRINPFPSFTSLRSLHWYMNAT*KRALKIGSG*CSKLHLVYSCITAIG*QRFNIE*QGTSGLNERPGKVRARYGRIKGPIERLLSHSLGGKCKTMVTASIHKTCPSVAQSVKCLSPLGPFPAYKRQVVSAFDATVDESASSAPHECCNVLHE*G*LRAVRCVAVLIVVDRPSRLENGTSIFGGSLTSMDCGVFTVLRLPIA*NPAGVKPSPLRDAACSETYTLHGFTAVRSESTKDTIELRSVPSTNLYPTPGACQLLGYHGACGSSRPISNFVMIKSPPREYQRR*LLSCA


#### 4. Сделать url запрос в GeneBank по идентификатору гена 7157, распарсить полученный файл, чтобы найти информацию: идентификатор в Uniprot кодируемого этим геном белка.

In [10]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = 'n.grechiha@gmail.com'
data = Entrez.efetch(db='gene', id='7157').read()

In [11]:
for c in ' \n':
    mod_data = data.lower().replace(c, '')
pos = mod_data.find('uniprot')
while pos != -1:
    sub = mod_data[pos:pos + 100]
    print(sub)
    print('/'*100)
    pos = mod_data.find('uniprot', pos+1)

Идентификатор в Uniprot кодируемого этим геном белка: p04637.

#### 5. Поиск мотивов в последовательности с помощью регулярных выражений (dna.txt).
Найти все мотивы (указать индексы начала мотивов в последовательности):

In [13]:
seq = open('dna.txt').read().strip()
from re import finditer

a)    GCNGC, где N означает любой нуклеотид – сайт, который находит BisI restriction enzyme.

In [14]:
pattern = r'GCNGC'.replace('N', '[ATGC]')
for x in finditer(pattern, seq):
    print(x.span(), x[0])

(296, 301) GCCGC
(580, 585) GCAGC
(1101, 1106) GCGGC
(1562, 1567) GCCGC
(1721, 1726) GCTGC
(1779, 1784) GCCGC


b)    Определить эукариотическую мРНК:

ATG – стартовый кодон.

За стартовым кодоном следует последовательность из нуклеотидов (A, T, G или C) длиной от 30 до 1000

Затем следует полиА-хвост (последовательность, состоящая только из А нуклеотидов) длиной от 5 до 10

In [15]:
pattern = r'ATG[ATGC]{30,1000}A{5,10}'
for x in finditer(pattern, seq):
    print(x.span(), x[0])

(102, 1066) ATGACATGCGTCTCCTTGCGGGTAAATCACCGACCGCAATTCATAGAAGCCTGGGGGAACAGATAGGTCTAATTAGCTTAAGAGAGTAAATCCTGGGATCATTCAGTAGTAACCATAAACTTACGCTGGGGCTTCTTCGGCGGATTTTTACAGTTACCAACCAGGAGATTTGAAGTAAATCAGTTGAGGATTTAGCCGCGCTATCCGGTAATCTCCAAATTAAAACATACCGTTCCATGAAGGCTAGAATTACTTACCGGCCTTTTCCATGCCTGCGCTATACCCCCCCACTCTCCCGCTTATCCGTCCGAGCGGAGGCAGTGCGATCCTCCGTTAAGATATTCTTACGTGTGACGTAGCTATGTATTTTGCAGAGCTGGCGAACGCGTTGAACACTTCACAGATGGTAGGGATTCGGGTAAAGGGCGTATAATTGGGGACTAACATAGGCGTAGACTACGATGGCGCCAACTCAATCGCAGCTCGAGCGCCCTGAATAACGTACTCATCTCAACTCATTCTCGGCAATCTACCGAGCGACTCGATTATCAACGGCTGTCTAGCAGTTCTAATCTTTTGCCAGCATCGTAATAGCCTCCAAGAGATTGATGATAGCTATCGGCACAGAACTGAGACGGCGCCGATGGATAGCGGACTTTCGGTCAACCACAATTCCCCACGGGACAGGTCCTGCGGTGCGCATCACTCTGAATGTACAAGCAACCCAAGTGGGCCGAGCCTGGACTCAGCTGGTTCCTGCGTGAGCTCGAGACTCGGGATGACAGCTCTTTAAACATAGAGCGGGGGCGTCGAACGGTCGAGAAAGTCATAGTACCTCGGGTACCAACTTACTCAGGTTATTGCTTGAAGCTGTACTATTTTAGGGGGGGAGCGCTGAAGGTCTCTTCTTCTCATGACTGAACTCGCGAGGGTCGTGAAGTCGGTTCCTTCAATGGTTAAAAAA
